# Lesson 8.2: Building Basic Graphs with LangGraph

---

In Lesson 8.1, we learned about the concept and importance of **LangGraph** in building complex, multi-step, stateful Agent applications. This lesson will dive into practical implementation, guiding you on how to **build a basic graph** in LangGraph, from defining the state to connecting processing steps (Nodes) with control flows (Edges).

## 1. Initializing a StateGraph and Defining the State Schema

Every graph in LangGraph starts with defining the **state** it will manage and pass between Nodes.

* **`StateGraph`:** Is the base class for building graphs. When initializing `StateGraph`, you need to pass a **State Schema**.
* **State Schema:**
    * Defines the data structure of the state.
    * Typically defined by inheriting from `TypedDict` (a Python feature for creating dictionaries with specific data types).
    * Each key in the `TypedDict` will be a field in the graph's state.
    * You also need to specify how new values for each field will be **merged** into the existing state (e.g., `operator.add` for string concatenation, `list.append` for appending to a list, or simply overwriting).

In [ ]:
from typing import TypedDict, Annotated, List, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph
import operator

# Define the state type for our graph
# Annotated[List[BaseMessage], operator.add] means:
# - chat_history is a list of BaseMessage objects.
# - When a Node returns an update for chat_history, it will be APPENDED (add) to the existing list,
#   not completely overwritten.
class AgentState(TypedDict):
    chat_history: Annotated[List[BaseMessage], operator.add]
    # You can add other fields to the state, e.g.:
    # current_query: str
    # tool_output: str
    # final_answer: str

# Initialize StateGraph with the defined state type
workflow = StateGraph(AgentState)

print("StateGraph initialized with AgentState.")

## 2. Defining Nodes (functions or LangChain Runnables) in the Graph

**Nodes** are the processing steps in your graph. Each Node receives the current graph state, performs some logic, and returns an update to that state.

* **Python Function:** A Node can be a regular Python function. This function must accept the current state as its first argument and return a dictionary containing state updates.
* **LangChain Runnable:** A Node can also be a `Runnable` (like an LLM, Chain, Agent) from LangChain. LangGraph will automatically handle passing the state in and receiving the output.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Node 1: Preprocessing input
def preprocess_node(state: AgentState) -> Dict[str, Any]:
    """
    This node takes chat_history, gets the latest message, and preprocesses it.
    Returns an update to the state.
    """
    print("--- Running Node: Preprocessing ---")
    last_message = state["chat_history"][-1]
    # Simulate preprocessing: convert Human message to a simple string
    processed_input = last_message.content.strip().lower()
    # Return a dictionary to update the state
    return {"chat_history": [HumanMessage(content=f"Processed: {processed_input}")]}

# Node 2: LLM Call
# We'll create a simple Chain to act as the LLM Node
llm_chain = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer the following question:"),
    ("human", "{question}"),
]) | llm | StrOutputParser()

def llm_node(state: AgentState) -> Dict[str, Any]:
    """
    This node calls the LLM with the preprocessed message.
    Returns the AI's response to update chat_history.
    """
    print("--- Running Node: LLM Call ---")
    # Get the preprocessed message (or original message)
    input_message = state["chat_history"][-1].content
    response = llm_chain.invoke({"question": input_message})
    # Return the AI's response to add to chat_history
    return {"chat_history": [AIMessage(content=response)]}

# Node 3: Postprocessing
def postprocess_node(state: AgentState) -> Dict[str, Any]:
    """
    This node receives the AI's response and performs postprocessing.
    Returns an update to the state.
    """
    print("--- Running Node: Postprocessing ---")
    last_ai_message = state["chat_history"][-1]
    # Simulate postprocessing: add an exclamation mark
    final_response = last_ai_message.content + " (Processed!)"
    # Update the last AI message in history
    # Note: If you want to overwrite the previous AI message, you need more complex logic
    # or redefine the merge behavior for chat_history.
    # Here, we will add a new message for illustration.
    return {"chat_history": [AIMessage(content=final_response)]}

print("Defined nodes: preprocess_node, llm_node, postprocess_node.")

## 3. Defining Edges (Control Flow) Between Nodes

**Edges** define the control flow between Nodes. They tell LangGraph which Node will run after which.

* **`add_node(name, node_function)`:** Adds a Node to the graph with a unique name and its corresponding function/Runnable.
* **`add_edge(start_node_name, end_node_name)`:** Adds a fixed edge from `start_node_name` to `end_node_name`. Control flow will always follow this edge.
* **`set_entry_point(node_name)`:** Sets the Node where the graph will begin execution.
* **`set_finish_point(node_name)`:** Sets the Node where the graph will end execution.

In [ ]:
# Add nodes to the workflow
workflow.add_node("preprocess", preprocess_node)
workflow.add_node("llm_process", llm_node)
workflow.add_node("postprocess", postprocess_node)

# Define sequential edges
workflow.add_edge("preprocess", "llm_process")
workflow.add_edge("llm_process", "postprocess")

# Set entry and finish points
workflow.set_entry_point("preprocess")
workflow.set_finish_point("postprocess")

print("Nodes and Edges added to workflow.")

## 4. Building a Simple Sequential Graph and Running It

After defining all Nodes and Edges, you need to "compile" the graph into a runnable `Runnable`.

* **`workflow.compile()`:** This method transforms the defined graph into a callable object, ready to receive input and run.

In [ ]:
# Compile the graph
app = workflow.compile()

print("Graph compiled.")

# Run the graph with an initial input
initial_state = {"chat_history": [HumanMessage(content="What's the weather like today?")]}
print("\n--- Running graph 1st time ---")
final_state = app.invoke(initial_state)

print("\n--- Final state of graph 1st run ---")
for message in final_state["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

# Run the graph a 2nd time with a different input
initial_state_2 = {"chat_history": [HumanMessage(content="What is LangGraph?")]}
print("\n--- Running graph 2nd time ---")
final_state_2 = app.invoke(initial_state_2)

print("\n--- Final state of graph 2nd run ---")
for message in final_state_2["chat_history"]:
    print(f"{message.type.capitalize()}: {message.content}")

## 5. Practical Example: Creating a Multi-step Text Processing Graph

We will use the above concepts to create a simple text processing graph:
**Preprocess -> LLM Process -> Postprocess**.

In [ ]:
# The entire code has been presented in the sections above.
# Below is how you can run the entire example seamlessly.

import os
from typing import TypedDict, Annotated, List, Dict, Any
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import StateGraph
import operator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# 1. Define the state type
class TextProcessingState(TypedDict):
    input_text: str
    processed_text: str
    llm_response: str
    final_output: str

# 2. Initialize StateGraph
text_workflow = StateGraph(TextProcessingState)

# 3. Initialize LLM
llm_for_text_processing = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

# 4. Define the Nodes
def initial_input_node(state: TextProcessingState) -> Dict[str, Any]:
    """Initial node, just takes input_text from the initial state."""
    print("--- Node: Initial Input ---")
    return {"input_text": state["input_text"]} # Ensure input_text is passed through

def preprocess_text_node(state: TextProcessingState) -> Dict[str, Any]:
    """Node for text preprocessing."""
    print("--- Node: Preprocessing Text ---")
    text = state["input_text"]
    processed_text = text.strip().replace("  ", " ").lower() # Example: remove extra spaces, convert to lowercase
    return {"processed_text": processed_text}

def llm_summarize_node(state: TextProcessingState) -> Dict[str, Any]:
    """Node that calls the LLM to summarize the preprocessed text."""
    print("--- Node: LLM Summarization ---")
    text_to_summarize = state["processed_text"]
    prompt_template = ChatPromptTemplate.from_template(
        "Summarize the following text concisely (around 20 words): {text}"
    )
    chain = prompt_template | llm_for_text_processing | StrOutputParser()
    summary = chain.invoke({"text": text_to_summarize})
    return {"llm_response": summary}

def postprocess_output_node(state: TextProcessingState) -> Dict[str, Any]:
    """Node for postprocessing the final output."""
    print("--- Node: Postprocessing Output ---")
    llm_response = state["llm_response"]
    final_output = f"Final Summary: \"{llm_response}\""
    return {"final_output": final_output}

# 5. Add Nodes to the graph
text_workflow.add_node("initial_input", initial_input_node)
text_workflow.add_node("preprocess_text", preprocess_text_node)
text_workflow.add_node("llm_summarize", llm_summarize_node)
text_workflow.add_node("postprocess_output", postprocess_output_node)

# 6. Define sequential Edges
text_workflow.add_edge("initial_input", "preprocess_text")
text_workflow.add_edge("preprocess_text", "llm_summarize")
text_workflow.add_edge("llm_summarize", "postprocess_output")

# 7. Set entry and finish points
text_workflow.set_entry_point("initial_input")
text_workflow.set_finish_point("postprocess_output")

# 8. Compile the graph
text_app = text_workflow.compile()

print("\n--- Practical: Running a Multi-step Text Processing Graph ---")
input_data = {"input_text": "   LangGraph is an extension   of LangChain. It helps build complex and stateful Agents.   "}
final_result = text_app.invoke(input_data)

print("\n--- Final State of Text Processing Graph ---")
print(f"Input Text: {final_result['input_text']}")
print(f"Processed Text: {final_result['processed_text']}")
print(f"LLM Response (Summary): {final_result['llm_response']}")
print(f"Final Output: {final_result['final_output']}")

print("\n--- End of Practical ---")